In [37]:
import time
import logging
from kiteconnect import KiteTicker
logger = logging.getLogger()
logging.basicConfig(level=logging.DEBUG)
import datetime
from utils.RenkoUtil import *
from classes.DayEntry import Day
from classes.MinuteEntry import Minute
from classes.TradeEntry import Trade
from classes.RenkoEntry import Renko
from collections import deque
import thread
# import csv
import pandas as pd

In [38]:
#initialize
money = 1
day = Day(0, money, False)
stepCount = 3
tickCount = 3
brickHeight = .15
renkoDeque = deque(maxlen=stepCount)
tickDeque = deque(maxlen=tickCount)
printRenkoDeque(renkoDeque)

DEBUG:root:empty queue


In [39]:
tickDeque.maxlen

3

In [40]:
df = pd.read_csv('tick_adani_15-04.csv')
# from utils.PreProcess import basicPreProcessing
# df = basicPreProcessing("IntradayData_2018", "ADANIENT")

In [41]:
df[:10]

,timestamp,lp_1
0,2019-05-15 10:48:08,119.25
1,2019-05-15 10:48:08,119.25
2,2019-05-15 10:48:09,119.25
3,2019-05-15 10:48:10,119.25
4,2019-05-15 10:48:11,119.25
5,2019-05-15 10:48:12,119.25
6,2019-05-15 10:48:13,119.25
7,2019-05-15 10:48:13,119.25
8,2019-05-15 10:48:14,119.25
9,2019-05-15 10:48:16,119.25


In [42]:
# newRenko = Renko(1242, 1243.2, -1, df['timestamp'][0])
# renkoDeque.append(newRenko)
# printRenkoDeque(renkoDeque)
def addToDeque(tickDeque, list_price):
    size = len(tickDeque)
    if size == 0:
        tickDeque.append(list_price)
    else:
        if size == tickDeque.maxlen:
            tickDeque.popleft()
        tickDeque.append(list_price)
    return tickDeque

def getAvg(tickDeque):
    size = len(tickDeque)
    if size == tickDeque.maxlen:
        return sum(tickDeque)/(1.0*size)
    else: 
        return None

In [43]:
def tickScript(tick):
    verbose = False
    global day
    global renkoDeque
    global stepCount
    global brickHeight
    global tickDeque
#     global tickCount
#     logging.info('Inside tickfunction that one')
    listPrice = tick['last_price']
    tickDeque = addToDeque(tickDeque, listPrice)
    ma_lp = getAvg(tickDeque)
    if not ma_lp:
        return
    minute = Minute(tick['timestamp'], ma_lp)
    renkoDeque, newRenko = generateRenko(minute, renkoDeque, brickHeight, stepCount, verbose)
    while newRenko:
#         printRenkoDeque(renkoDeque)
#         print('\n')
        if len(renkoDeque) == stepCount:
    #             if marketOpen <= minute.time < marketClose:
            logging.info("Boughtflag: {}".format(day.boughtFlag))
            if day.boughtFlag == 0:
                #market enter logic
                if getEmotion(renkoDeque) == stepCount:
                    #buy
                    day.boughtFlag = 1
                    day.currentTrade = Trade(listPrice, minute.time, 0, 0, day.boughtFlag)
                    logging.info('Long trade {}'.format(day.currentTrade.enterPrice))

                elif getEmotion(renkoDeque) == -1 * stepCount:
                    #sell
                    day.boughtFlag = -1
                    day.currentTrade  = Trade(listPrice, minute.time, 0, 0, day.boughtFlag)
                    logging.info('Short trade {}'.format(day.currentTrade.enterPrice))

            elif day.boughtFlag == 1:
                #market exit logic (when already bought)
                if renkoExitLogic(renkoDeque, stepCount, day.boughtFlag):
                    logging.info('exiting trade, enter: {} exit: {}'.format(day.currentTrade.enterPrice, listPrice))
                    day.currentTrade.exitTrade('renkoExit', minute.time, listPrice, verbose = True)
                    day.addTrade(day.currentTrade)
                    logging.info('Long exit')
            else:
                #market exit logic (when already sold)
                if renkoExitLogic(renkoDeque, stepCount, day.boughtFlag):
                    logging.info('exiting trade, enter: {} exit: {}'.format(day.currentTrade.enterPrice, listPrice))
                    day.currentTrade.exitTrade('renkoExit', minute.time, listPrice, verbose = True)
                    day.addTrade(day.currentTrade)
                    logging.info('Short exit')
        renkoDeque, newRenko = generateRenko(minute, renkoDeque, brickHeight, stepCount, verbose)
    #         elif minute.time >= marketClose and day.sellEndOfDay:
    #             if day.boughtFlag == 1:
    #change to currentTrade
    #                 trade.exitTrade('eod', minute.time, minute.closingPrice, verbose)
    #                 day.addTrade(trade)
    #             elif day.boughtFlag == -1:
    #                 trade.exitTrade('eod', minute.time, minute.closingPrice, verbose)
    #                 day.addTrade(trade)

In [44]:
for row in df.iterrows():
    tick = {}
    tick['timestamp'] = row[1].timestamp
    tick['last_price'] = row[1].lp_1
#     logging.debug("Ticks: {} {}".format(tick['timestamp'], tick['last_price']))
    try:
        tickScript(tick)
    except:
        logging.debug('Unable to start that one thread')
#     time.sleep(0.1)


INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Short trade 118.7
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -1
DEBUG:root:renkoExit : True
INFO:root:exiting trade, enter: 118.7 exit: 119.2
DEBUG:root:Inside exitTrade fn
DEBUG:root:Inside addTrade to day fn
DEBUG:root:money 0.995606208054
DEBUG:root:after reset bought flag : 0
INFO:root:Short exit
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Long trade 119.6


-1, 118.7000, 119.2000, -0.4394
2019-05-15 10:48:42 2019-05-15 10:49:58



INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 1
DEBUG:root:renkoExit : True
INFO:root:exiting trade, enter: 119.6 exit: 119.4
DEBUG:root:Inside exitTrade fn
DEBUG:root:Inside addTrade to day fn
DEBUG:root:money 0.993742526466
DEBUG:root:after reset bought flag : 0
INFO:root:Long exit
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Short trade 119.25
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -1
DEBUG:root:renkoExit : True
INFO:root:exiting trade, enter: 119.25 exit: 119.25
DEBUG:root:Inside exitTrade fn
DEBUG:root:Inside addTrade to day fn
DEBUG:root:money 0.993543777961
DEBUG:root:after reset bought flag : 0
INFO:root:Short exit
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Short trade 118.6
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3


1, 119.6000, 119.4000, -0.1872
2019-05-15 11:02:56 2019-05-15 11:06:28

-1, 119.2500, 119.2500, -0.0200
2019-05-15 11:06:32 2019-05-15 11:07:41



INFO:root:Boughtflag: -1
DEBUG:root:emotion : -1
DEBUG:root:renkoExit : True
INFO:root:exiting trade, enter: 118.6 exit: 118.9
DEBUG:root:Inside exitTrade fn
DEBUG:root:Inside addTrade to day fn
DEBUG:root:money 0.990838731773
DEBUG:root:after reset bought flag : 0
INFO:root:Short exit
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0


...

-1, 117.4000, 117.9000, -0.4440
2019-05-15 14:58:05 2019-05-15 15:01:18

-1, 117.1500, 117.4500, -0.2754
2019-05-15 15:06:41 2019-05-15 15:08:35



INFO:root:Boughtflag: 1
DEBUG:root:emotion : 1
DEBUG:root:renkoExit : True
INFO:root:exiting trade, enter: 117.75 exit: 117.4
DEBUG:root:Inside exitTrade fn
DEBUG:root:Inside addTrade to day fn
DEBUG:root:money 0.951727885567
DEBUG:root:after reset bought flag : 0
INFO:root:Long exit
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Short trade 117.05
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -1
DEBUG:root:renkoExit : True
INFO:root:exiting trade, enter: 117.05 exit: 117.45
DEBUG:root:Inside exitTrade fn
DEBUG:root:Inside addTrade to day fn
DEBUG:root:money 0.948296884256


1, 117.7500, 117.4000, -0.3172
2019-05-15 15:10:44 2019-05-15 15:13:09

-1, 117.0500, 117.4500, -0.3605
2019-05-15 15:22:49 2019-05-15 15:24:36



DEBUG:root:after reset bought flag : 0
INFO:root:Short exit
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Short trade 116.7
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -3
INFO:root:Boughtflag: -1
DEBUG:root:emotion : -1
DEBUG:root:renkoExit : True
INFO:root:exiting trade, enter: 116.7 exit: 117.0
DEBUG:root:Inside exitTrade fn
DEBUG:root:Inside addTrade to day fn
DEBUG:root:money 0.945676180712
DEBUG:root:after reset bought flag : 0
INFO:root:Short exit
INFO:root:Boughtflag: 0
INFO:root:Boughtflag: 0
INFO:root:Long trade 117.4
INFO:root:Boughtflag: 1
DEBUG:root:emotion : 3


-1, 116.7000, 117.0000, -0.2764
2019-05-15 15:28:55 2019-05-15 15:30:00



In [45]:
day.money

0.9456761807123002

In [46]:
day.boughtFlag

1

In [47]:
day.dailyTrades

23

In [48]:
day.printAllTrades(True)

-1, 118.7000, 119.2000, -0.4394
2019-05-15 10:48:42 2019-05-15 10:49:58

1, 119.6000, 119.4000, -0.1872
2019-05-15 11:02:56 2019-05-15 11:06:28

-1, 119.2500, 119.2500, -0.0200
2019-05-15 11:06:32 2019-05-15 11:07:41

-1, 118.6000, 118.9000, -0.2723
2019-05-15 11:15:21 2019-05-15 11:22:27

-1, 118.2500, 118.5000, -0.2309
2019-05-15 11:48:20 2019-05-15 11:49:10

-1, 117.8500, 118.2000, -0.3160
2019-05-15 11:51:08 2019-05-15 11:51:39

1, 118.5500, 118.1500, -0.3573
2019-05-15 11:53:08 2019-05-15 12:01:19

1, 118.6500, 118.3000, -0.3149
2019-05-15 12:16:36 2019-05-15 12:20:03

-1, 118.0000, 117.6000, 0.3201
2019-05-15 12:45:13 2019-05-15 12:47:45

1, 117.9500, 117.7000, -0.2319
2019-05-15 12:49:40 2019-05-15 13:06:10

-1, 117.2500, 117.5000, -0.2327
2019-05-15 13:16:35 2019-05-15 13:17:19

1, 117.7500, 117.5500, -0.1898
2019-05-15 13:21:50 2019-05-15 13:24:45

1, 118.0500, 117.7000, -0.3164
2019-05-15 13:34:08 2019-05-15 13:39:00

1, 118.5000, 118.1500, -0.3153
2019-05-15 13:51:02 2019-05